### Projected spending on most costly brand name medicines in English primary care given US Medicare Part D Prices, 2018

Import Statements

In [1]:
import pandas as pd
import numpy as np
from ebmdatalab import bq

This is the SQL which is run against BigQuery. We select here the BNF name, the total cost, total items, and total quantity from Open Prescribing database between 2018 Jan to 2018 Dec. We group by BNF code and name and then we order data based on spending 

In [2]:
#SQL 2018 UK prescriptions based on quantity 

sql = """

SELECT 
  bnf_code,
  TRIM(bnf_name) as bnf_name,
  sum(actual_cost) as sum_cost,
  sum(items) as sum_items,
  sum(quantity) as sum_quantity
FROM 
  hscic.normalised_prescribing_standard 
where (month >= "2018-01-01" AND month <= "2018-12-31") 
GROUP BY
  bnf_code, 
  bnf_name
ORDER BY
  sum(actual_cost) DESC
  
"""

UK_df = bq.cached_read(sql, csv_path='UK_df.zip', use_cache=True)


In [3]:
UK_df.head()

,bnf_code,bnf_name,sum_cost,sum_items,sum_quantity
0,0208020Z0AAABAB,Apixaban_Tab 5mg,1.394279e+08,2895195,158067469
1,0208020Y0AAACAC,Rivaroxaban_Tab 20mg,1.391612e+08,2855911,83267272
2,0302000C0BQAABX,Fostair_Inh 100mcg/6mcg (120D) CFF,1.065387e+08,3312807,3914881
3,0601023X0AAAAAA,Sitagliptin_Tab 100mg,7.053167e+07,2047796,63930397
4,0208020Z0AAAAAA,Apixaban_Tab 2.5mg,6.705012e+07,1654909,75915952


We add the chemical column for each row in the dataframe by taking the first 9 digits of the BNF code

In [4]:
#add chemical_code column

UK_df['chemical_code'] = UK_df['bnf_code'].str[:9]

In [5]:
UK_df.head()

,bnf_code,bnf_name,sum_cost,sum_items,sum_quantity,chemical_code
0,0208020Z0AAABAB,Apixaban_Tab 5mg,1.394279e+08,2895195,158067469,0208020Z0
1,0208020Y0AAACAC,Rivaroxaban_Tab 20mg,1.391612e+08,2855911,83267272,0208020Y0
2,0302000C0BQAABX,Fostair_Inh 100mcg/6mcg (120D) CFF,1.065387e+08,3312807,3914881,0302000C0
3,0601023X0AAAAAA,Sitagliptin_Tab 100mg,7.053167e+07,2047796,63930397,0601023X0
4,0208020Z0AAAAAA,Apixaban_Tab 2.5mg,6.705012e+07,1654909,75915952,0208020Z0


This is the SQL which is run against BigQuery. We select here the chemical code and chemical name from the bnf database. This is to facilitate brand name screening later. 

In [6]:
#obtain chemical_code and chemical_name from SQL (bnf database)

sql = """

SELECT DISTINCT
    chemical_code,
    chemical
FROM 
  hscic.bnf   

"""

chemical_df = bq.cached_read(sql, csv_path='chemical.zip', use_cache=True)


In [7]:
chemical_df.head()

,chemical_code,chemical
0,010101000,Other Antacid & Simeticone Preps
1,0101010A0,Alexitol Sodium
2,0101010B0,Almasilate
3,0101010C0,Aluminium Hydroxide
4,0101010D0,Aluminium Hydroxide With Magnesium


Here we merge the original and second dataframes to obtain a merged dataframe - so that the chemical name is attached each item in the original dataframe 

In [8]:
#merge the UK and chemical code dataframes 

UK_merge = pd.merge(UK_df,
                 chemical_df,
                 on='chemical_code', how = "left")

In [9]:
UK_merge.head()

,bnf_code,bnf_name,sum_cost,sum_items,sum_quantity,chemical_code,chemical
0,0208020Z0AAABAB,Apixaban_Tab 5mg,1.394279e+08,2895195,158067469,0208020Z0,Apixaban
1,0208020Y0AAACAC,Rivaroxaban_Tab 20mg,1.391612e+08,2855911,83267272,0208020Y0,Rivaroxaban
2,0302000C0BQAABX,Fostair_Inh 100mcg/6mcg (120D) CFF,1.065387e+08,3312807,3914881,0302000C0,Beclometasone Dipropionate
3,0601023X0AAAAAA,Sitagliptin_Tab 100mg,7.053167e+07,2047796,63930397,0601023X0,Sitagliptin
4,0208020Z0AAAAAA,Apixaban_Tab 2.5mg,6.705012e+07,1654909,75915952,0208020Z0,Apixaban


Here we obtain the sum of the cost of each item prescribed based on chemical code and name. This will allow for the identification of the most costly chemicals/medicines in the UK in 2018

In [10]:
#sum quantity based on chemical 

chemical_sum_cost = UK_merge.groupby(['chemical_code', 'chemical'])['sum_cost', 'sum_items', 'sum_quantity'].sum().reset_index()

In [11]:
chemical_sum_cost.head()

,chemical_code,chemical,sum_cost,sum_items,sum_quantity
0,0101010C0,Aluminium Hydroxide,59966.66165,5657,561344
1,0101010F0,Magnesium Carbonate,90660.07159,413,30674
2,0101010G0,Co-Magaldrox(Magnesium/Aluminium Hydrox),213256.61420,62425,36457932
3,0101010I0,Magnesium Oxide,538351.65549,4164,447427
4,0101010J0,Magnesium Trisilicate,55637.28221,4333,1060455


In [12]:
#sort based on cost

sorted_cost = chemical_sum_cost.sort_values(by = ['sum_cost'], ascending = [False]).reset_index()

In [13]:
sorted_cost.head()

,index,chemical_code,chemical,sum_cost,sum_items,sum_quantity
0,1173,090402000,Enteral Nutrition,2.284312e+08,3872902,19163525531
1,321,0302000C0,Beclometasone Dipropionate,2.143523e+08,10653368,12552913
2,266,0208020Z0,Apixaban,2.069046e+08,4560045,234467170
3,323,0302000N0,Fluticasone Propionate (Inh),1.994316e+08,5155648,6616076
4,265,0208020Y0,Rivaroxaban,1.801875e+08,3912605,107997262


Here, we export the dataframe to excel for manual screening. To facilitate screening, we first exclude chemicals that are not medicines that would not be included in Medicare Part D (e.g. food stuffs, vitamins, and over-the-counter medication). We then manually screen for chemicals that only exist as brand name in the US and UK. 

For the US, we used the FDA Orange Book - which has data on all medicines approved by the FDA and their corresponding patent expiry dates. We used Drugs@FDA to identify drugs that are not available in the US (but available in the UK). 

For the UK, we used the PCA data to identify which drugs were brand-name-only before the end of 2018. 


The FDA Orange Book can be found here: https://www.accessdata.fda.gov/scripts/cder/ob/index.cfm 
The Drugs@FDA can be found here: https://www.accessdata.fda.gov/scripts/cder/daf/
The PCA data can be found here: https://openprescribing.net/long_term_trends/


In [14]:
sorted_cost.to_excel('data_for_screening.xlsx')